In [ ]:
! pip install awscli boto3 optuna imbalanced-learn mlflow

In [ ]:
! aws configure

In [9]:
import mlflow
# Step 2 Set Mlflow Tracking URI
mlflow.set_tracking_uri('http://ec2-100-26-36-125.compute-1.amazonaws.com:5000/')

In [10]:
# Set Experiment
mlflow.set_experiment('E ML Algos with HP Tuning')

<Experiment: artifact_location='s3://om-mlflow-bucket/622937322449081937', creation_time=1741368134336, experiment_id='622937322449081937', last_update_time=1741368134336, lifecycle_stage='active', name='E ML Algos with HP Tuning', tags={}>

In [11]:
import mlflow.sklearn
import optuna
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

In [12]:
df=pd.read_csv('/content/reddit_processed.csv').dropna()
df.shape

(36662, 2)

In [13]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Random Forest

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

X = df['clean_comment']
y = df['category']

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 10000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Random Forest
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)  # Number of trees in the forest
    max_depth = trial.suggest_int('max_depth', 3, 20)  # Maximum depth of the tree
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)  # Minimum samples required to split a node
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)  # Minimum samples required at a leaf node

    # RandomForestClassifier setup
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                   min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                   random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Random Forest, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_rf, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                                        max_depth=best_params['max_depth'],
                                        min_samples_split=best_params['min_samples_split'],
                                        min_samples_leaf=best_params['min_samples_leaf'],
                                        random_state=42)

    # Log the best model with MLflow, passing the algo_name as "RandomForest"
    log_mlflow("RandomForest", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Random Forest
run_optuna_experiment()


[I 2025-03-07 17:50:32,232] A new study created in memory with name: no-name-f7a62552-1904-4c68-a1cf-4f331f9c5408
[I 2025-03-07 17:50:38,216] Trial 0 finished with value: 0.6866221191872358 and parameters: {'n_estimators': 248, 'max_depth': 17, 'min_samples_split': 2, 'min_samples_leaf': 11}. Best is trial 0 with value: 0.6866221191872358.
[I 2025-03-07 17:50:40,701] Trial 1 finished with value: 0.6783035592526933 and parameters: {'n_estimators': 131, 'max_depth': 16, 'min_samples_split': 2, 'min_samples_leaf': 19}. Best is trial 0 with value: 0.6866221191872358.
[I 2025-03-07 17:50:44,768] Trial 2 finished with value: 0.6559389063139234 and parameters: {'n_estimators': 244, 'max_depth': 10, 'min_samples_split': 17, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.6866221191872358.
[I 2025-03-07 17:50:45,681] Trial 3 finished with value: 0.6252556934406109 and parameters: {'n_estimators': 51, 'max_depth': 6, 'min_samples_split': 16, 'min_samples_leaf': 17}. Best is trial 0 with va

🏃 View run RandomForest_SMOTE_TFIDF_Trigrams at: http://ec2-100-26-36-125.compute-1.amazonaws.com:5000/#/experiments/622937322449081937/runs/3c092b6a9de044aea5101b32b3ef770c
🧪 View experiment at: http://ec2-100-26-36-125.compute-1.amazonaws.com:5000/#/experiments/622937322449081937
